In [ ]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 4.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
import pandas as pd
import json
from sentence_transformers import SentenceTransformer, util

In [ ]:
# Sample data frame
data = pd.read_csv("mails-for-emb.csv")
data.head()

,file,content-to-emb
0,data/maildir/love-p/personal/30.,"Subject: !!. From: Morse, Brad Sent: Tuesday, ..."
1,data/maildir/love-p/sent_items/776.,"Subject: !!. From: Mills, Bruce Sent: Tuesday,..."
2,data/maildir/salisbury-h/inbox/196.,Subject: !!!!!!!!!!!GONE.SCR VIRUS Warning!!!!...
3,data/maildir/salisbury-h/inbox/194.,Subject: !!!!!!!!!!!GONE.SCR VIRUS Warning!!!!...
4,data/maildir/scott-s/inbox/402.,"Subject: ""'Hot"" Natural Gas curves - status as..."


In [ ]:
data.shape

(79805, 2)

In [ ]:
# Initialize SentenceTransformer model
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v1')

# Function to embed paragraphs
def embed_paragraph(paragraph):
    return model.encode(paragraph).tolist()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.90k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
model.max_seq_length

512

In [ ]:
# Apply embedding function to the 'paragraph' column
data['embedding'] = data['content-to-emb'].apply(embed_paragraph)

data.head()

,file,content-to-emb,embedding
0,data/maildir/love-p/personal/30.,"Subject: !!. From: Morse, Brad Sent: Tuesday, ...","[-0.032669853419065475, 0.02763822302222252, -..."
1,data/maildir/love-p/sent_items/776.,"Subject: !!. From: Mills, Bruce Sent: Tuesday,...","[0.06922703236341476, 0.0302390456199646, 0.01..."
2,data/maildir/salisbury-h/inbox/196.,Subject: !!!!!!!!!!!GONE.SCR VIRUS Warning!!!!...,"[0.02881448343396187, 0.003450637450441718, 0...."
3,data/maildir/salisbury-h/inbox/194.,Subject: !!!!!!!!!!!GONE.SCR VIRUS Warning!!!!...,"[0.007789650000631809, -0.03071964718401432, 0..."
4,data/maildir/scott-s/inbox/402.,"Subject: ""'Hot"" Natural Gas curves - status as...","[-0.03331587463617325, -0.0007603323902003467,..."


In [ ]:
# Compute embedding for both lists
embeddings1 = data.loc[0,'embedding']
embeddings2 = data.loc[1,'embedding']
cosine_scores = util.cos_sim(embeddings1, embeddings2)
cosine_scores

tensor([[0.6403]])

In [ ]:
cosine_scores

tensor([[0.5125]])

In [ ]:
# Compute embedding for both lists
embeddings1 = data.loc[2,'embedding']
embeddings2 = data.loc[1,'embedding']
cosine_scores = util.cos_sim(embeddings1, embeddings2)
cosine_scores

tensor([[0.2902]])

In [ ]:
file_path = '/content/drive/My Drive/mails-embeddings.csv'
data.to_csv(file_path, index=False)

In [ ]:
embedding_dict = dict(zip(data['file'], data['embedding']))

# Save the dictionary to a JSON file
file_path = '/content/drive/My Drive/mails-embeddings.json'
with open(file_path, 'w') as json_file:
    json.dump(embedding_dict, json_file)